In [8]:

import pickle
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
from joblib import Parallel, delayed


#load static arbitrary graph
with open("fixed_graph_arbitrary.pkl", "rb") as f:
    G = pickle.load(f)
#verify structure
print(f"Loaded arbitrary graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges and {np.mean([d for _, d in G.degree()])} average degree.")

#populate A_initial from graph metadata
A_initial = np.array([G.nodes[n]["initial_opinion"] for n in G.nodes])
lambda_val = 0.5  
N = A_initial.shape[0]
l = A_initial.shape[1]


Loaded arbitrary graph with 50 nodes and 532 edges and 21.28 average degree.


In [10]:

#computing clustering coefficient and centrality measures
clustering_coeffs = nx.clustering(G)
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

#converting centrality and clustering metrics to a frame for analysis
network_metrics = pd.DataFrame({
    "Node": list(G.nodes()),
    "Degree": [G.degree(n) for n in G.nodes()],
    "Clustering Coefficient": [clustering_coeffs[n] for n in G.nodes()],
    "Degree Centrality": [degree_centrality[n] for n in G.nodes()],
    "Betweenness Centrality": [betweenness_centrality[n] for n in G.nodes()]
})


print(network_metrics.head())
network_metrics.to_csv("[DIRECTORY]/network_metrics2playerP1first.csv", index=False)



Network Metrics (First 5 Rows):
   Node  Degree  Clustering Coefficient  Degree Centrality  \
0     0      21                0.438095           0.428571   
1     1      22                0.588745           0.448980   
2     2      19                0.473684           0.387755   
3     3      13                0.564103           0.265306   
4     4      18                0.555556           0.367347   

   Betweenness Centrality  
0                0.014192  
1                0.008854  
2                0.010102  
3                0.002889  
4                0.006347  

Average Connection Probability Per Topic:
                    Avg Probability
affirmative_action         0.807715
gun_laws                   0.711856
party_id                   0.747058


In [11]:
#Player class and Simulation
class Player:
    def __init__(self, theta_low, theta_high, delta, beta, topic_index, agent_id, push_toward, l, kappa, gamma):
        self.theta_low = theta_low
        self.theta_high = theta_high
        self.delta = delta
        self.beta = beta
        self.topic_index = topic_index
        self.agent_id = agent_id
        self.push_toward = push_toward
        self.l = l
        self.kappa = kappa
        self.gamma = gamma

    def compute_w_link(self, A_updated):
        N = A_updated.shape[0]
        w_link_matrix = np.zeros((N, N))
        for i in range(N):
            for j in range(i + 1, N):
                distance = np.linalg.norm(A_updated[i] - A_updated[j])
                w_link_matrix[i, j] = np.exp(-distance)
                w_link_matrix[j, i] = w_link_matrix[i, j]
        return w_link_matrix

    def send_message(self, network, initial_node, message, opponent_message, A_initial):
        active_nodes = set()
        claimed_nodes = {}
        A_updated = np.copy(A_initial)

        if opponent_message is None:
            opponent_message = np.full(self.l, 0)

        alignment_self = self.beta * (1 - np.linalg.norm(A_initial[initial_node] - message) / (2 * np.sqrt(self.l)))
        alignment_opponent = self.beta * (1 - np.linalg.norm(A_initial[initial_node] - opponent_message) / (2 * np.sqrt(self.l)))

        if alignment_self > alignment_opponent and alignment_self > self.theta_low:
            w_link_matrix = self.compute_w_link(A_updated)
            neighbors = list(network.neighbors(initial_node))
            Z_i = np.sum(w_link_matrix[initial_node, neighbors])
            neighbor_influence = np.sum([A_updated[j] * w_link_matrix[initial_node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0
            A_updated[initial_node] = (1 - self.kappa - self.gamma) * A_updated[initial_node] + self.kappa * message + self.gamma * neighbor_influence
            active_nodes.add(initial_node)
            claimed_nodes[initial_node] = self.agent_id

        while True:
            new_active_nodes = set()
            w_link_matrix = self.compute_w_link(A_updated)
            for node in active_nodes:
                for neighbor in network.neighbors(node):
                    if neighbor in claimed_nodes:
                        continue
                    if w_link_matrix[node, neighbor] > self.delta:
                        alignment_self = self.beta * (1 - np.linalg.norm(A_updated[neighbor] - message) / (2 * np.sqrt(self.l)))
                        alignment_opponent = self.beta * (1 - np.linalg.norm(A_updated[neighbor] - opponent_message) / (2 * np.sqrt(self.l)))
                        if alignment_self > alignment_opponent and alignment_self > self.theta_low:
                            neighbors = list(network.neighbors(node))
                            Z_i = np.sum(w_link_matrix[node, neighbors])
                            neighbor_influence = np.sum([A_updated[j] * w_link_matrix[node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0
                            A_updated[neighbor] = (1 - self.kappa - self.gamma) * A_updated[neighbor] + self.kappa * message + self.gamma * neighbor_influence
                            new_active_nodes.add(neighbor)
                            claimed_nodes[neighbor] = self.agent_id
            if not new_active_nodes:
                break
            active_nodes.update(new_active_nodes)

        total_influence = self.push_toward * np.sum(A_updated[:, self.topic_index] - A_initial[:, self.topic_index])
        return total_influence, A_updated, claimed_nodes

    def optimize_strategy(self, network, opponent_message, A_initial):
        best_node, best_message, max_influence = None, None, -float('inf')
        base_vectors = [np.eye(self.l)[i] for i in range(self.l)] + [-np.eye(self.l)[i] for i in range(self.l)]
        sampled_messages = [v for v in base_vectors] + [np.random.uniform(-1, 1, self.l) for _ in range(3)]

        for initial_node in network.nodes:
            for message in sampled_messages:
                influence, _, _ = self.send_message(network, initial_node, message, opponent_message, A_initial)
                if influence > max_influence:
                    max_influence = influence
                    best_node = initial_node
                    best_message = message

        if best_node is None:
            best_node = np.random.choice(list(network.nodes))
            best_message = np.ones(self.l) if self.push_toward == 1 else -1 * np.ones(self.l)

        return best_node, best_message, max_influence

def simulate_diffusion(player, network, initial_node, message, A_initial):
    active_nodes_history = []
    A_updated = np.copy(A_initial)
    active_nodes = set([initial_node])
    claimed_nodes = {initial_node: player.agent_id}
    active_nodes_history.append(set(active_nodes))

    while True:
        new_active_nodes = set()
        w_link_matrix = player.compute_w_link(A_updated)
        for node in active_nodes:
            for neighbor in network.neighbors(node):
                if neighbor in claimed_nodes:
                    continue
                if w_link_matrix[node, neighbor] > player.delta:
                    alignment = player.beta * (1 - np.linalg.norm(A_updated[neighbor] - message) / (2 * np.sqrt(player.l)))
                    if alignment > player.theta_low:
                        neighbors = list(network.neighbors(node))
                        Z_i = np.sum(w_link_matrix[node, neighbors])
                        neighbor_influence = np.sum([A_updated[j] * w_link_matrix[node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0
                        A_updated[neighbor] = (1 - player.kappa - player.gamma) * A_updated[neighbor] + player.kappa * message + player.gamma * neighbor_influence
                        new_active_nodes.add(neighbor)
                        claimed_nodes[neighbor] = player.agent_id
        if not new_active_nodes:
            break
        active_nodes.update(new_active_nodes)
        active_nodes_history.append(set(active_nodes))

    return active_nodes_history, A_updated, claimed_nodes



In [20]:
#simulation wrapper for parallel
def run_two_player_simulation(sim, topic_index, G, A_initial, player_config):
    player1 = Player(topic_index=topic_index, agent_id=1, push_toward=1, l=A_initial.shape[1], **player_config)
    player2 = Player(topic_index=topic_index, agent_id=2, push_toward=-1, l=A_initial.shape[1], **player_config)

    n1, m1, infl1 = player1.optimize_strategy(G,  None, A_initial)
    n2, m2, infl2 = player2.optimize_strategy(G,  m1, A_initial)

    hist1, A1, c1 = simulate_diffusion(player1, G, n1, m1, A_initial)
    hist2, A2, c2 = simulate_diffusion(player2, G, n2, m2, A_initial)

    #visualize_diffusion_as_gif(G, hist1, hist2, A1, A2, c1, c2, topic_index, m1, m2)

    return [
        {"Simulation": sim + 1, "Topic": topic_index, "Agent": 1, "Optimal Node": n1, "Total Influence": infl1, "Optimal Message": m1, "Nodes Influenced": len(c1)},
        {"Simulation": sim + 1, "Topic": topic_index, "Agent": 2, "Optimal Node": n2, "Total Influence": infl2, "Optimal Message": m2, "Nodes Influenced": len(c2)}
    ]

#function for parallel for multi simulation
def run_parallel_two_player_simulations(G, A_initial, num_simulations=5, topics=[0, 1, 2]):
    player_config = {
        "theta_low": 0.3,
        "theta_high": 0.7,
        "delta": 0.4,
        "beta": 1,
        "kappa": 0.3,
        "gamma": 0.2
    }

    jobs = [(sim, topic, G, A_initial, player_config) for sim in range(num_simulations) for topic in topics]

    results_nested = Parallel(n_jobs=-1)(
        delayed(run_two_player_simulation)(*job) for job in jobs
    )

    results = [entry for sublist in results_nested for entry in sublist]
    df_results = pd.DataFrame(results)
    df_results.to_csv("[DIRECTORY]/2player_parallel_simulation_resultsP1First.csv", index=False)
    print("Saved to 2player_parallel_simulation_resultsP1First.csv")
    return df_results

In [21]:
df_results=run_parallel_two_player_simulations(G, A_initial, num_simulations=25, topics=[0, 1, 2])

Saved to 2player_parallel_simulation_resultsP1First.csv
